# Import

In [ ]:
import tensorflow as tf
from tensorflow import keras
!pip install -q -U keras-tuner
import keras_tuner as kt

     |████████████████████████████████| 98 kB 3.7 MB/s 


# Loading Data

In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

4431872/4422102 [==============================] - 0s 0us/step


# Base Model

In [ ]:
def model_builder():
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  model.add(keras.layers.Dense(units=512, activation='relu'))
  model.add(keras.layers.Dense(10))
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [ ]:
model = model_builder()
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)


Epoch 1/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.5757 - accuracy: 0.8018 - val_loss: 0.4972 - val_accuracy: 0.8244
Epoch 2/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.4489 - accuracy: 0.8410 - val_loss: 0.4291 - val_accuracy: 0.8552
Epoch 3/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4139 - accuracy: 0.8515 - val_loss: 0.4100 - val_accuracy: 0.8563
Epoch 4/50
1500/1500 [==============================] - 7s 4ms/step - loss: 0.4204 - accuracy: 0.8500 - val_loss: 0.4940 - val_accuracy: 0.8365
Epoch 5/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.4005 - accuracy: 0.8569 - val_loss: 0.4443 - val_accuracy: 0.8446
Epoch 6/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3888 - accuracy: 0.8602 - val_loss: 0.4200 - val_accuracy: 0.8577
Epoch 7/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3800 - accuracy: 0.8655 - val_loss: 0.4463 - val_accuracy:

# Keras Tuner

In [ ]:
def model_builder(hp):
  dense_neurons = hp.Int(name='dense_neurons', min_value=16, max_value=512, step=16)
  learning_rate = hp.Choice('learning_rate', values=[0.0001, 0.0005, 0.001, 0.005, 0.01])
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  model.add(keras.layers.Dense(units=dense_neurons, activation='relu'))
  model.add(keras.layers.Dense(10))
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [ ]:
tuner = kt.Hyperband(
    hypermodel=model_builder,
    max_epochs=10,
    hyperband_iterations=1,
    factor=3,
    directory='./Q4',
    objective='val_accuracy',
)
tuner.search_space_summary()

Search space summary
Default search space size: 2
dense_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 512, 'step': 16, 'sampling': None}
learning_rate (Choice)
{'default': 0.0001, 'conditions': [], 'values': [0.0001, 0.0005, 0.001, 0.005, 0.01], 'ordered': True}


In [ ]:
tuner.search(img_train, label_train, batch_size=128, validation_split=0.2)

Trial 30 Complete [00h 00m 15s]
val_accuracy: 0.8915833234786987

Best val_accuracy So Far: 0.8915833234786987
Total elapsed time: 00h 04m 05s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in ./Q4/untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
dense_neurons: 352
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.8915833234786987
Trial summary
Hyperparameters:
dense_neurons: 432
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: eaf52b95623c107b745c4e3bc4c95a98
Score: 0.8865000009536743
Trial summary
Hyperparameters:
dense_neurons: 224
learning_rate: 0.0005
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.8823333382606506
Trial summary
Hyperparameters:
dense_neurons: 240
learning_rate: 0.005
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.8815833330154419
Trial summary
Hyperparameters:
dense_neurons: 272
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
tuner/round: 1
tuner/trial

# train best model

In [ ]:
best_hp = tuner.get_best_hyperparameters(1)[0]
best_model = model_builder(best_hp)
best_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_2 (Dense)             (None, 352)               276320    
                                                                 
 dense_3 (Dense)             (None, 10)                3530      
                                                                 
Total params: 279,850
Trainable params: 279,850
Non-trainable params: 0
_________________________________________________________________


In [ ]:
best_model.fit(img_train, label_train, epochs=50, validation_split=0.2, batch_size=128)

Epoch 1/50
375/375 [==============================] - 2s 4ms/step - loss: 0.5422 - accuracy: 0.8115 - val_loss: 0.4220 - val_accuracy: 0.8550
Epoch 2/50
375/375 [==============================] - 1s 4ms/step - loss: 0.3970 - accuracy: 0.8593 - val_loss: 0.3888 - val_accuracy: 0.8647
Epoch 3/50
375/375 [==============================] - 1s 4ms/step - loss: 0.3540 - accuracy: 0.8732 - val_loss: 0.3561 - val_accuracy: 0.8720
Epoch 4/50
375/375 [==============================] - 1s 4ms/step - loss: 0.3253 - accuracy: 0.8817 - val_loss: 0.3688 - val_accuracy: 0.8672
Epoch 5/50
375/375 [==============================] - 1s 4ms/step - loss: 0.3028 - accuracy: 0.8900 - val_loss: 0.3279 - val_accuracy: 0.8843
Epoch 6/50
375/375 [==============================] - 1s 4ms/step - loss: 0.2859 - accuracy: 0.8969 - val_loss: 0.3209 - val_accuracy: 0.8819
Epoch 7/50
375/375 [==============================] - 1s 4ms/step - loss: 0.2700 - accuracy: 0.9004 - val_loss: 0.3144 - val_accuracy: 0.8864
Epoch 